In [21]:
%pip install -r requirements.txt

   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   -------- ------------------------------- 2.1/9.9 MB 10.7 MB/s eta 0:00:01
   ------------------- -------------------- 4.7/9.9 MB 11.4 MB/s eta 0:00:01
   ---------------------------- ----------- 7.1/9.9 MB 11.5 MB/s eta 0:00:01
   ------------------------------------- -- 9.2/9.9 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 10.4 MB/s eta 0:00:00

   ---------------------------------------- 0/3 [patsy]
   ---------------------------------------- 0/3 [patsy]
   ------------- -------------------------- 1/3 [statsmodels]
   ------------- -------------------------- 1/3 [statsmodels]
   ------------- -------------------------- 1/3 [statsmodels]
   ------------- -------------------------- 1/3 [statsmodels]
   ------------- -------------------------- 1/3 [statsmodels]
   ------------- -------------------------- 1/3 [statsmodels]
   ------------- -------------------------- 1/3 [statsmodels]

In [5]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

import warnings
warnings.filterwarnings('ignore')

In [16]:
load_dotenv()

True

In [44]:
import requests
from bs4 import BeautifulSoup
import json

def get_all_shoes():
    # Создаем сессию для сохранения cookies между запросами
    session = requests.Session()
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Referer": "https://www.velostrana.ru/velosipedy/",
        "Origin": "https://www.velostrana.ru"
    }
    
    # Первый запрос для получения начальных cookies
    init_response = session.get("https://www.velostrana.ru/velosipedy/", headers=headers)
    if init_response.status_code != 200:
        print(f"Ошибка начального запроса: {init_response.status_code}")
        return []
    
    # Устанавливаем количество элементов на странице
    settings_url = "https://www.velostrana.ru/catalog/settings/"
    settings_headers = headers.copy()
    settings_headers["Content-Type"] = "application/x-www-form-urlencoded; charset=UTF-8"
    
    settings_response = session.post(
        settings_url,
        headers=settings_headers,
        data={"per_page": "3000"}
    )
    
    if settings_response.status_code != 200:
        print(f"Ошибка при изменении настроек: {settings_response.status_code}")
        return []
    
    # Получаем велосипеды с новыми настройками
    shoes_url = "https://www.velostrana.ru/velosipedy/"
    shoes_headers = headers.copy()
    shoes_headers["Content-Type"] = "application/x-www-form-urlencoded; charset=UTF-8"
    
    shoes_response = session.post(shoes_url, headers=shoes_headers, data={})
    
    if shoes_response.status_code != 200:
        print(f"Ошибка при получении велосипедов: {shoes_response.status_code}")
        return []
    
    try:
        shoes_data = shoes_response.json()
    except json.JSONDecodeError:
        print("Ошибка декодирования JSON")
        return []
    
    if "html" not in shoes_data:
        print("В ответе отсутствует HTML")
        return []
    
    return parse_shoes_html(shoes_data["html"])

def parse_shoes_html(html):
    soup = BeautifulSoup(html, "html.parser")
    bike_cards = soup.find_all("div", class_="product-card")
    
    shoes = []
    
    for card in bike_cards:
        try:
            bike = {
                "name": card.find("div", class_="product-card__model").get_text(strip=True),
                "brand": card.find("meta", itemprop="brand")["content"],
                "category": card.find("div", class_="product-card__category").get_text(strip=True),
                "price": card.find("div", class_="product-card__price").get_text(strip=True).replace("\u20bd", "").strip(),
                "old_price": None,
                "discount": None,
                "in_stock": card.find("div", class_="product-card__instock").get_text(strip=True),
                "url": "https://www.velostrana.ru" + card.find("a", class_="product-card__title")["href"],
                "image_url": None,
                "description": []
            }
            
            # Проверяем наличие старой цены и скидки
            discount_div = card.find("div", class_="product-card__discount")
            if discount_div:
                old_price = discount_div.find("div", class_="product-card__oldprice")
                if old_price:
                    bike["old_price"] = old_price.get_text(strip=True)
                
                sale = discount_div.find("div", class_="product-card__sale")
                if sale:
                    bike["discount"] = sale.get_text(strip=True)
            
            # Получаем URL изображения
            img_tag = card.find("img", itemprop="image") or card.find("img")
            if img_tag:
                bike["image_url"] = img_tag.get("src", "").replace("medium.jpg", "big.jpg")
            
            # Получаем описание
            desc = card.find("ul", class_="product-card__desc")
            if desc:
                bike["description"] = [li.get_text(strip=True) for li in desc.find_all("li")]
            
            shoes.append(bike)
            
        except Exception as e:
            print(f"Ошибка при парсинге карточки велосипеда: {e}")
            continue
    
    return shoes

In [ ]:
all_shoes = get_all_shoes()
print(f"Всего найдено велосипедов: {len(all_shoes)}")

# Выводим первые 3 велосипеда для примера
for i, bike in enumerate(all_shoes[:3], 1):
    print(f"\nВелосипед #{i}:")
    for key, value in bike.items():
        print(f"{key}: {value}")

# Сохраняем все данные в JSON файл
with open("shoes.json", "w", encoding="utf-8") as f:
    json.dump(all_shoes, f, ensure_ascii=False, indent=2)

print("\nВсе данные сохранены в файл 'shoes.json'")

Ошибка при парсинге карточки велосипеда: 'NoneType' object has no attribute 'get_text'
Ошибка при парсинге карточки велосипеда: 'NoneType' object has no attribute 'get_text'
Всего найдено велосипедов: 2998

Велосипед #1:
name: Aspect Radium Pro 29 (2025)
brand: Aspect
category: Горный велосипед
price: 45 990руб
old_price: None
discount: None
in_stock: В наличии
url: https://www.velostrana.ru/aspect/radium-pro-29/
image_url: https://cdn.velostrana.ru/upload/models/velo/66071/full/11568be7350f3acc11fc7d2ae4617848.jpg
description: ['Горный велосипед', 'Модель оснащена алюминиевой рамой', 'Дисковые гидравлические тормоза', '8 скоростей', 'Пружинно-эластомерная с блокировкой вилка', 'Задний переключатель Shimano Tourney']

Велосипед #2:
name: Aspect Air 29 (2025)
brand: Aspect
category: Горный велосипед
price: 76 990руб
old_price: 84 490
discount: -9%
in_stock: В наличии
url: https://www.velostrana.ru/aspect/air-29/
image_url: https://cdn.velostrana.ru/upload/models/velo/66188/big.jpg
descr

In [15]:
# Инициализация модели эмбедера
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

In [ ]:
import json
import psycopg
from tqdm import tqdm

# Подключение к БД
conn = psycopg.connect(
    dbname="shoes",
    user="postgres",
    host="localhost",
    port="5430"
)
cur = conn.cursor()

# Загрузка данных
with open("shoes.json", "r", encoding="utf-8") as f:
    shoes = json.load(f)

def process_price(price_text):
    """Обработка цены с защитой от ошибок"""
    if not price_text:
        return None
    try:
        # Удаляем все нецифровые символы, кроме точки
        cleaned = ''.join(c for c in price_text if c.isdigit() or c == '.')
        return float(cleaned) if cleaned else None
    except (ValueError, TypeError):
        return None

for bike in tqdm(shoes):
    # Склеивание описания
    full_desc = ". ".join(bike["description"])
    
    # Генерация эмбеддинга
    embedding = model.encode([full_desc])[0].tolist()
    
    # Вставка данных
    cur.execute("""
        INSERT INTO shoes (
            name, brand, category, price, old_price, discount, 
            in_stock, url, image_url, full_description, 
            embedding, fts_vector
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
            to_tsvector('russian', %s)
        )
    """, (
        bike["name"],
        bike["brand"],
        bike["category"],
        process_price(bike["price"]),
        bike["old_price"],
        bike["discount"],
        bike["in_stock"],
        bike["url"],
        bike["image_url"],
        full_desc,
        embedding,
        full_desc  # Для полнотекстового индекса
    ))

conn.commit()
cur.close()
conn.close()

100%|██████████| 2998/2998 [24:09<00:00,  2.07it/s]


In [16]:
from pgvector.psycopg import register_vector

def connect_db():
    conn = psycopg.connect(
        dbname="shoes",
        user="postgres",
        host="localhost",
        port="5430"
    )
    register_vector(conn)
    return conn

def vector_search(query: str, limit: int = 5):
    """Поиск по векторному сходству"""
    conn = connect_db()
    cur = conn.cursor()
    
    # Генерация эмбеддинга для запроса
    query_embedding = model.encode([query], prompt_name="query")[0]
    
    cur.execute("""
        SELECT name, full_description, brand, price, url,
            1 - (embedding <=> %s) AS similarity
        FROM shoes
        ORDER BY embedding <=> %s
        LIMIT %s
    """, (query_embedding, query_embedding, limit))
    
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

def fulltext_search(query: str, limit: int = 5):
    """Полнотекстовый поиск"""
    conn = connect_db()
    cur = conn.cursor()
    
    cur.execute("""
        SELECT name, full_description, brand, price, url,
            ts_rank(fts_vector, websearch_to_tsquery('russian', %s)) AS rank
        FROM shoes
        WHERE fts_vector @@ websearch_to_tsquery('russian', %s)
        ORDER BY rank DESC
        LIMIT %s
    """, (query, query, limit))
    
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

def hybrid_search(query: str, limit: int = 5):
    """Гибридный поиск (векторный + полнотекстовый)"""
    conn = connect_db()
    cur = conn.cursor()
    
    # Генерация эмбеддинга
    query_embedding = model.encode([query], prompt_name="query")[0]
    
    cur.execute("""
        SELECT name, full_description, brand, price, url,
            (0.7 * (1 - (embedding <=> %s)) + 
             0.3 * ts_rank(fts_vector, websearch_to_tsquery('russian', %s))) AS score
        FROM shoes
        ORDER BY score DESC
        LIMIT %s
    """, (query_embedding, query, limit))
    
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

In [ ]:
# Поиск по вектору
print("Vector search results:")
for result in vector_search("горный велосипед с гидравлическими тормозами"):
    print(f"{result[5]:.2f} | {result[0]} {result[3]} ({result[1]})")

# Полнотекстовый поиск
print("\nFulltext search results:")
for result in fulltext_search("9 скоростей"):
    print(f"{result[5]:.2f} | {result[0]} {result[3]} ({result[1]})")

# Гибридный поиск
print("\nHybrid search results:")
for result in hybrid_search("алюминиевая рама 8 скоростей электровелик"):
    print(f"{result[5]:.2f} | {result[0]} {result[3]} ({result[1]})")

Vector search results:


NameError: name 'vector_search' is not defined

In [7]:
import plotly.express as px
import plotly.subplots as sp
from sklearn.cluster import KMeans
from umap import UMAP
from pgvector.psycopg import register_vector
import numpy as np
import psycopg


# Загрузка данных (как в предыдущем примере)
def load_data():
    conn = psycopg.connect(dbname="shoes", user="postgres", host="localhost", port="5430")
    register_vector(conn)
    cur = conn.cursor()
    cur.execute("SELECT name, brand, category, price, embedding FROM bikes WHERE embedding IS NOT NULL")
    data = cur.fetchall()
    cur.close()
    conn.close()
    
    metadata = {
        'names': [x[0] for x in data],
        'brands': [x[1] for x in data],
        'categories': [x[2] for x in data],
        'prices': [x[3] for x in data]
    }
    embeddings = np.array([x[4] for x in data])
    return metadata, embeddings

def visualize_umap_kmeans(embeddings, metadata=None):
    # UMAP снижение размерности
    umap_reducer = UMAP(n_components=2, metric='cosine', random_state=42)
    umap_embeddings = umap_reducer.fit_transform(embeddings)
    
    # KMeans кластеризация
    kmeans = KMeans(n_clusters=4, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    
    # Создаем DataFrame для визуализации
    fig = px.scatter(
        x=umap_embeddings[:, 0],
        y=umap_embeddings[:, 1],
        color=clusters.astype(str),
        title='UMAP + KMeans кластеризация',
        labels={'color': 'Кластер'},
        hover_name=metadata['names'] if metadata else None,
        hover_data={
            'brand': metadata['brands'] if metadata else None,
            'category': metadata['categories'] if metadata else None,
            'price': metadata['prices'] if metadata else None
        } if metadata else None
    )
    
    fig.update_layout(
        xaxis_title='UMAP 1',
        yaxis_title='UMAP 2',
        showlegend=True
    )
    
    fig.show()

In [8]:
metadata, embeddings = load_data()
visualize_umap_kmeans(embeddings, metadata)

In [9]:
llm = ChatOpenAI(
    model="deepseek-chat",
    max_tokens=10000,
    temperature=0.7,
    top_p=0.8,
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("BASE_URL"),
    extra_body={"response_format": {"type": "json_object"}}
)

In [11]:
print(llm.invoke("Что такое велосипед? Ответь в формате JSON {\"answer\":...}").content)

{
  "answer": "Велосипед - это транспортное средство, приводимое в движение мускульной силой человека через педали или (редко) через ручные рычаги. Он имеет два колеса, расположенных одно за другим, и предназначен для передвижения по различным поверхностям. Велосипеды используются для перевозки людей, грузов, а также в спортивных и рекреационных целях."
}
